In [1]:
!java -version

java version "22.0.2" 2024-07-16
Java(TM) SE Runtime Environment (build 22.0.2+9-70)
Java HotSpot(TM) 64-Bit Server VM (build 22.0.2+9-70, mixed mode, sharing)


In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

NameError: name 'os' is not defined

In [3]:
!pip install -q tabula-py
!pip install selenium requests 

In [4]:
!pip install selenium requests beautifulsoup4
!pip install PyMuPDF

In [5]:
import os
import re
import tabula
import pandas as pd
import numpy as np
import fitz  # module from PyMuPDF

In [6]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [7]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [12]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

DOWNLOAD_DIR = './pdf_downloads'
BASE_URL = 'https://anteco.com.ph/index.php/disclosures'

# Setup Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless Chrome (no GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920x1080')

driver = webdriver.Chrome(options=options)

try:
    driver.get(BASE_URL)

    # Wait for the page to fully load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, 'a'))
    )

    # Extract all links to PDF files
    pdf_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.pdf')]")
    
    # Prepare the download directory
    os.makedirs(DOWNLOAD_DIR, exist_ok=True)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/pdf'
    }

    for link in pdf_links:
        try:
            href = link.get_attribute('href')
            pdf_name = href.split('/')[-1]
            full_url = href

            # Download the PDF
            response = requests.get(full_url, headers=headers)
            if response.status_code == 200:
                pdf_path = os.path.join(DOWNLOAD_DIR, pdf_name)
                with open(pdf_path, 'wb') as file:
                    file.write(response.content)
                print(f'Downloaded: {pdf_name}')
            else:
                print(f'Failed to download {pdf_name}, status code: {response.status_code}')
        
        except Exception as e:
            print(f'Error processing link: {e}')

finally:
    driver.quit()


Downloaded: BR202407.pdf
Downloaded: bdgc-202407.pdf
Downloaded: BR202406.pdf
Downloaded: bdgc-202406.pdf
Downloaded: BR202405.pdf
Downloaded: bdgc-202405.pdf
Downloaded: BR202404.pdf
Downloaded: bdgc-202404.pdf
Downloaded: BR202403.pdf
Downloaded: bdgc-202403.pdf
Downloaded: BR202402.pdf
Downloaded: bdgc-202402.pdf
Downloaded: BR202401.pdf
Downloaded: bdgc-202401.pdf
Downloaded: BR202312.pdf
Downloaded: bdgc-202312.pdf
Downloaded: BR202311.pdf
Downloaded: bdgc-202311.pdf
Downloaded: BR202310.pdf
Downloaded: bdgc-202310.pdf
Downloaded: BR202309.pdf
Downloaded: bdgc-202309.pdf
Downloaded: BR202308.pdf
Downloaded: bdgc-202308.pdf
Downloaded: BR202307.pdf
Downloaded: bdgc-202307.pdf
Downloaded: BR202306.pdf
Downloaded: bdgc-202306.pdf
Downloaded: BR202305.pdf
Downloaded: bdgc-202305.pdf
Downloaded: BR202304.pdf
Downloaded: bdgc-202304.pdf
Downloaded: BR202303.pdf
Downloaded: bdgc-202303.pdf
Downloaded: BR202302.pdf
Downloaded: bdgc-202302.pdf
Downloaded: BR202301.pdf
Downloaded: bdgc-2023

In [15]:
# Process the downloaded PDFs into DataFrames
import tabula

dataframes = {}

# List all PDF files in the download directory
pdf_files = [f for f in os.listdir(DOWNLOAD_DIR) if f.endswith('.pdf')]

for pdf_file in pdf_files:
    date_part = pdf_file.split('_')[-1].replace('.pdf', '')
    pdf_path = os.path.join(DOWNLOAD_DIR, pdf_file)

    try:
        # Read the PDF file into a DataFrame
        dfs = tabula.read_pdf(pdf_path, stream=True, pages='all')  # Adjust pages parameter as needed
        if dfs:  # Check if any DataFrames were returned
            df = dfs[0]
            dataframes[f"df_{date_part}"] = df
            print(f'Loaded DataFrame for: {pdf_file}')
        else:
            print(f'No data found in {pdf_file}')
    except Exception as e:
        print(f'Error reading {pdf_file}: {e}')

# Now you can use the `dataframes` dictionary to ac

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path/to/your/download_dir'